In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import keras
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-06-09 18:12:13.803655: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-09 18:12:14.560480: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-09 18:12:14.560953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = Image.open(img_path).convert('L')  # Conversion en gris
        img = Image.open(img_path).convert('RGB')  # Conversion en gris
        img = img.resize((128, 128))  # Resize les images
        img_array = np.array(img, dtype=np.float64).flatten()  # Flatten les vecteurs
        images.append(img_array)
        labels.append(label)
    return images, labels

folder_a = 'data/dataset/resized/gas_giant'
folder_b = 'data/dataset/resized/neptune-like'
folder_c = 'data/dataset/resized/super_earth'

images_a, labels_a = load_images_from_folder(folder_a, [1.0, 0.0, 0.0])
images_b, labels_b = load_images_from_folder(folder_b, [0.0, 1.0, 0.0])
images_c, labels_c = load_images_from_folder(folder_c, [0.0, 0.0, 1.0])

X = np.array(images_a + images_b + images_c)
y = np.array(labels_a + labels_b + labels_c)

# Séparer les données entrainement / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Training data shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test data shape: {y_test.shape}")

Training data shape: (1200, 49152)
Training data shape: (1200, 3)
Test data shape: (300, 49152)
Test data shape: (300, 3)


In [8]:

# Définir le modèle MLP
model = Sequential([
    Flatten(input_shape=(128, 128, 3)),  # Aplatir les images en vecteurs
    Dense(32, activation='softmax'),
    Dense(3, activation='softmax')  # Softmax pour la couche de sortie
])

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [9]:
# Entraîner le modèle avec le mini-batch gradient descent
history = model.fit(X_train, y_train, 
                    epochs=50,  # Nombre d'époques
                    batch_size=32,  # Taille du batch
                    validation_data=(X_test, y_test))  # Données de validation


Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 49152), dtype=float32). Expected shape (None, 128, 128, 3), but input has incompatible shape (None, 49152)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 49152), dtype=float32)
  • training=True
  • mask=None

In [ ]:
# Évaluer le modèle sur les données de test
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


In [ ]:
import matplotlib.pyplot as plt

# Tracer la courbe de perte
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Tracer la courbe de précision
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
